# Refinery

Hosts a web API for refining subject definitions created in the RecordThing App.



### Refinery Request

```json
{
  "subject": "string",
  "definition": "string"
}
```

## DINOv2 Test case

In [1]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs[0]

# We have to force return_dict=False for tracing
model.config.return_dict = False

with torch.no_grad():
    traced_model = torch.jit.trace(model, [inputs.pixel_values])
    traced_outputs = traced_model(inputs.pixel_values)

print((last_hidden_states - traced_outputs[0]).abs().max())

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/transformers/modeling_utils.py:4728: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/transformers/models/dinov2/modeling_dinov2.py:164: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:


tensor(0., grad_fn=<MaxBackward1>)


## Refinery API hosting

In [1]:
import asyncio, tornado

class MainHandler(tornado.web.RequestHandler):
    def get(self):
        self.write("Hello, world")
        
async def main():
    app = tornado.web.Application([
        (r"/", MainHandler),
    ])
    app.listen(8889)
    asyncio.get_event_loop().stop()
    await asyncio.get_event_loop().run_forever()

await main()

RuntimeError: This event loop is already running

: 

In [1]:
from tornado.platform.asyncio import AsyncIOMainLoop
await AsyncIOMainLoop().install()

RuntimeError: IOLoop <tornado.platform.asyncio.AsyncIOMainLoop object at 0x103a17b10> already associated with asyncio loop <_UnixSelectorEventLoop running=True closed=False debug=False>

In [8]:
import time,asyncio

async def count():
    print("count one")
    await asyncio.sleep(1)
    print("count four")

async def count_further():
    print("count two")
    await asyncio.sleep(1)
    print("count five")

async def count_even_further():
    print("count three")
    await asyncio.sleep(1)
    print("count six")

async def main():
    await asyncio.gather(count(), count_further(), count_even_further())

s = time.perf_counter()
await main()
elapsed = time.perf_counter() - s
print(f"Script executed in {elapsed:0.2f} seconds.")

count one
count two
count three
count four
count five
count six
Script executed in 1.00 seconds.
